In [1]:
import pandas as pd
import getpass
from sqlalchemy import create_engine
from sqlalchemy.exc import ResourceClosedError
import MySQLdb 
check = False
import json

In [2]:
class MySQLConnection:
    def __init__(self,user,hostname,db):
        self.user = str(user)
        self.hostname = str(hostname)
        self.db = str(db)
        pword = getpass.getpass("Enter password for user {}".format(user))
        # self.cnx = mysql.connector.connect(host=self.hostname,user=self.user, password=pword,database=self.db)
        self.engine = create_engine("mysql+mysqlconnector://{}:{}@{}/{}".format(user,pword,hostname,db))
        self.cnx = self.engine.connect()
    
    def write_to_db(self, df, table_name):
        try:
            df.to_sql(table_name,con = self.cnx,if_exists='fail',index=False)
        except Exception as e:
            print("\n SQL Write error with: ")
            print(df,"\n", e)
    
    def query(self,q):
        try:
            res = pd.read_sql(q,self.cnx)
            return res
        except ResourceClosedError:
            print("Query success, nothing was returned")

In [3]:
sql = MySQLConnection('ubuntu','localhost','classicmodels')

Enter password for user ubuntu········


In [4]:
sql.query("show tables")

,Tables_in_classicmodels
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


In [5]:
# sql.query("create table orderdetails_w_income as select *, priceeach*quantityordered as income from orderdetails")

In [13]:
ienakumi = """
select 
    sum(income) as kopejie_ienakumi,
    city
from 
    customers join 
    (
    select
        orders.customernumber,
        orderdetails_w_income.income
    from 
        orderdetails_w_income join orders on
        orderdetails_w_income.ordernumber = orders.ordernumber
    ) pasutijumu_ienakumi on 
    customers.customernumber = pasutijumu_ienakumi.customernumber
group by
    city
order by 
    kopejie_ienakumi desc
"""

sql.query(ienakumi)

,kopejie_ienakumi,city
0,979880.77,Madrid
1,591827.34,San Rafael
2,497941.50,NYC
3,386514.63,Auckland
4,263997.78,Singapore
...,...,...
72,42570.37,Brisbane
73,41506.19,Los Angeles
74,31310.09,Munich
75,29217.18,Charleroi


In [17]:
sql.query("SELECT status, count(*) as statusi from orders group by status")

,status,statusi
0,Cancelled,6
1,Disputed,3
2,In Process,6
3,On Hold,4
4,Resolved,4
5,Shipped,303


In [10]:
# Uzdevums:
# Sastādiet klientu sarakstu ar kuriem būtu jāsazinās, lai atjaunotu kādu iztrūkstošu personīgo informāciju
q_1 = """
select * 
from
    customers
where 
    phone is NULL or (addressline1 is NULL and addressline2 is NULL) 
"""


sql.query(q_1)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit


In [15]:
# Uzdevums: 
# 1. Atlasiet kopējos ienākumus no visiem pasūtījumiem, 
# 2. Atlasiet kopējos ienākumus tikai no pasūtījumiem, kas ir piegādāti (shipped)
# q2 = """
# select
#     sum(amount) as summarie_ienakumi
# from
#     payments
# """

# q3 = """
# select 
#     sum(amount) as summarie_shipped_ienakumi
# from (
#           select * 
#           from
#               orders
#           where
#               status = 'shipped'
#      ) orders_shipped join 
#      (
#          select 
#              customers.customernumber,
#              payments.amount
#          from 
#              customers join payments on
#              customers.customernumber = payments.customernumber
#      ) customerpayments on 
#      orders_shipped.customernumber = customerpayments.customernumber
# """

q3 = """
select 
    sum(amount) as summarie_shipped_ienakumi
from (
          select * 
          from
              orders
          where
              status = 'shipped'
     ) orders_shipped join payments on
     orders_shipped.customernumber = payments.customernumber
"""
sql.query(q3)


,summarie_shipped_ienakumi
0,47693096.63


In [13]:
q3_2 = """
select 
    sum(amount)
from 
    payments join orders on
    payments.customernumber = orders.customernumber
where
    status = 'shipped'
"""
sql.query(q3_2)

,sum(amount)
0,47693096.63


In [23]:
# Uzdevums:
# 1. Atlasiet 10 visvairāk pārdotās preces
top_sold = """
select 
    productcode,
    sum(quantityordered) as products_ordered
from
    (
        select
            productcode,
            quantityordered 
        from 
            orderdetails
    ) details
group by 
    productcode
order by 
    products_ordered desc
limit 
    10
"""
sql.query(top_sold)

# 2. Izpētiet datus un izdomājiet pie kādiem nosacījumiem preču skaits būtu jāpapildina 
# (pieņemsim, ka pārdošanai pieejamo preču skaits ir papildināms par jebkādu skaitu 2 dienās)
# 3. Atlasiet preces, kuras būtu jāiepērk pēc šiem izvēlētajiem kritērijiem

,productcode,products_ordered
0,S18_3232,1808.0
1,S18_1342,1111.0
2,S700_4002,1085.0
3,S18_3856,1076.0
4,S50_1341,1074.0
5,S18_4600,1061.0
6,S10_1678,1057.0
7,S12_4473,1056.0
8,S18_2319,1053.0
9,S24_3856,1052.0


In [26]:
sql.query("select * from employees")

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,NaN,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002.0,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002.0,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056.0,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056.0,Sale Manager (EMEA)
5,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056.0,Sales Manager (NA)
6,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143.0,Sales Rep
7,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143.0,Sales Rep
8,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143.0,Sales Rep
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep


In [31]:
# Uzdevums:
# Uzrakstiet SQL vaicājumu, kurš saskaita cik ir darbinieku, kuriem ir padotie
management_count = """
select
    count(*) as management_count
from 
    employees inner join 
    (
    select 
        distinct reportsto as reports 
    from employees
    ) emps
    on employees.employeenumber = emps.reports
"""

management_count = """select count(distinct reportsto) from employees """

sql.query(management_count)

,count(distinct reportsto)
0,6


In [36]:
sql.query("select * from payments where customernumber = 103")

,customerNumber,checkNumber,paymentDate,amount
0,103,HQ336336,2004-10-19,6066.78
1,103,JM555205,2003-06-05,14571.44
2,103,OM314933,2004-12-18,1676.14


In [39]:
latest_set = """
select 
    customernumber,
    max(paymentdate)
from 
    payments
group by
    customernumber
"""
sql.query(latest_set)

,customernumber,max(paymentdate)
0,103,2004-12-18
1,112,2004-12-17
2,114,2004-12-15
3,119,2005-02-22
4,121,2004-11-28
...,...,...
93,486,2004-11-23
94,487,2004-02-29
95,489,2004-01-31
96,495,2004-05-14


In [44]:
# Uzdevums:
# Uzrakstiet SQL vaicājumu, kurš katram customer id piekārto visnesenāko maksājumu

latest_payments = """
select 
    max_dates.customernumber,
    checknumber,
    paymentdate,
    amount
from 
    payments join 
    (
    select 
        customernumber,
        max(paymentdate) as max_date
    from 
        payments
    group by
        customernumber
    ) max_dates on 
    max_dates.customernumber = payments.customernumber
where 
    max_date = paymentdate
"""

sql.query(latest_payments)
# Katram customer var būt vairāki pirkumi.
# sql.query("select customernumber, count(*) as pirkumi from payments group by customernumber order by pirkumi desc")

,customernumber,checknumber,paymentdate,amount
0,103,OM314933,2004-12-18,1676.14
1,112,BO864823,2004-12-17,14191.12
2,114,MA765515,2004-12-15,82261.22
3,119,NG94694,2005-02-22,49523.67
4,121,MA302151,2004-11-28,34638.14
...,...,...,...,...
93,486,HS86661,2004-11-23,45994.07
94,487,PT550181,2004-02-29,12573.28
95,489,PO860906,2004-01-31,7310.42
96,495,FN155234,2004-05-14,6276.60


In [21]:
# Uzdevums: 
# Uzrakstiet python funkciju, kuras input ir valīds produkta kods un output ir json teksta pavediens, 
# kas satur lauka nosaukumu un informāciju par preci.
# Piem. ja funkcijas input būtu S700_3505, 
# tad funkcijai būtu jāatgriež sekojošs teksta pavediens (piemērā sadalīts vairākās rindiņās lasāmībai):


# {
# "product_description":"Completed model measures 19 1\\/2 inches long, 9 inches high, 3inches 
# wide and is in barn red\\/black. All wood and metal.",
# "product_name":"The Titanic",
# "original_price": 51.09,
# "quantity_in_stock":1956
# }


In [ ]:
# Uzdevums:
# Papildiniet šo funkciju ar funkcionalitāti, kas json pavedienā, string mainīgo vietā atgriež "default" un skaitļu vietā 0, 
# ja nav atrasts neviens produkts ar šādu produkta kodu